In [42]:
path = "/Users/az/Downloads/raport.pdf"

# tabula-py

In [ ]:
!pip install tabula-py

In [ ]:
import tabula

# Read PDF file and extract tables into a list of DataFrames
tables = tabula.read_pdf(path, pages='1')

# Check the number of tables extracted
print("Total tables extracted:", len(tables))

# Print the first table as a Pandas DataFrame
print(tables[0])

# Export the first table to CSV
# tabula.convert_into('example.pdf', 'extracted_table.csv', output_format='csv', pages='1')

In [9]:
tables[3]

,27-02-2024 Wt.


# camelot-py

In [ ]:
!pip install camelot-py opencv-python ghostscript
!brew install ghostscript tcl-tk

In [1]:
import camelot

In [8]:
# Read PDF file
tables = camelot.read_pdf(path, pages='5')

# Check the number of tables extracted
print("Total tables extracted:", tables.n)

# Print the first table as a Pandas DataFrame
print(tables)

# Export the table to CSV
# tables[0].to_csv('extracted_table.csv')

Total tables extracted: 0
<TableList n=0>


# Tesseract

In [11]:
!tesseract image.png output_text.txt

Detected 9 diacritics


# LayoutPDFReader

https://github.com/nlmatics/llmsherpa#layoutpdfreader

LayoutPDFReader solves this problem by parsing PDFs along with hierarchical layout information such as:
- Sections and subsections along with their levels.
- Paragraphs - combines lines.
- Links between sections and paragraphs.
- Tables along with the section the tables are found in.
- Lists and nested lists.
- Join content spread across pages.
- Removal of repeating headers and footers.
- Watermark removal.

In [ ]:
!pip install llmsherpa

In [ ]:
from llmsherpa.readers import LayoutPDFReader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(path)
# TimeoutError

# LlamaParse

LlamaParse is the world's first genAI-native document parsing platform, 1k free pages a day, as of March 2024

Src
- https://www.llamaindex.ai/blog/launching-the-first-genai-native-document-parsing-platform
- https://medium.com/@salujav4/parsing-pdfs-text-image-and-tables-for-rag-based-applications-using-llamaparse-llamaindex-0f4c5ed50fb7
- https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/
- https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_nodes/
- https://github.com/run-llama/llama_parse/blob/main/examples/demo_advanced.ipynb

In [ ]:
!pip install llama-parse

In [30]:
import os
from dotenv import load_dotenv
load_dotenv()
LLAMA_CLOUD_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")

In [34]:
import nest_asyncio
nest_asyncio.apply()
from llama_parse import LlamaParse

parser = LlamaParse(
    api_key=LLAMA_CLOUD_API_KEY,
    result_type="markdown"
)

documents = await parser.aload_data(path)

Started parsing the file under job_id ed512035-7831-4108-ad44-fbc115c0693d


In [39]:
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.llms.openai import OpenAI

node_parser = MarkdownElementNodeParser(\
    llm=OpenAI(model="gpt-4o"), 
    num_workers=4,
    language="pl"
)
nodes = node_parser.get_nodes_from_documents(documents=[documents[0]])
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

1it [00:00, 3214.03it/s]

# Unstructured

The unstructured library provides open-source components for ingesting and pre-processing images and text documents, such as PDFs, HTML, Word docs, and many more.

https://github.com/Unstructured-IO/unstructured

BEST RESULT

In [ ]:
!brew install poppler-utils libmagic-dev tesseract-ocr tesseract-lang
!pip install 'unstructured[all-docs]' nltk

In [ ]:
from unstructured.partition.auto import partition

elements = partition(filename=path)
print("\n\n".join([str(el) for el in elements]))